In [0]:
## Raccomandation System

In [0]:
## 2 type of raccomandation system , 
## Content Based 
## Collaborative Filtering 

In [0]:
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.appName('rec').getOrCreate()

In [0]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [0]:
data = spark.read.csv('/FileStore/tables/movielens_ratings.csv',inferSchema=True,header=True)

In [0]:
data.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [0]:
# Smaller dataset so we will use 0.8 / 0.2
(training, test) = data.randomSplit([0.8, 0.2])

In [0]:
# Build the recommendation model using ALS on the training data
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating")
model = als.fit(training)

In [0]:
# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)

In [0]:
predictions.show()

+-------+------+------+------------+
|movieId|rating|userId|  prediction|
+-------+------+------+------------+
|      3|   1.0|     0|  0.71740484|
|      2|   2.0|     1|   1.0869545|
|      0|   1.0|     3|  0.87416685|
|      1|   1.0|     3|   0.7960733|
|      1|   1.0|     4|   0.8680023|
|      0|   1.0|     5|  0.26344982|
|      0|   1.0|     6|  0.69684464|
|      2|   2.0|     7|   2.0512388|
|      0|   1.0|     8|    2.277235|
|      2|   4.0|    10|  -1.7446228|
|      0|   1.0|    11|-0.036684018|
|      0|   1.0|    13|  0.23747781|
|      3|   1.0|    13|   1.0756879|
|      1|   4.0|    15|  0.85285497|
|      2|   1.0|    15|    1.083394|
|      2|   1.0|    19|  0.99508315|
|      1|   1.0|    20|  0.17844084|
|      2|   4.0|    21|    3.543792|
|      0|   1.0|    22|   1.0422769|
|      1|   3.0|    25|  0.33275145|
+-------+------+------+------------+
only showing top 20 rows



In [0]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 1.8387978690908393


In [0]:
single_user = test.filter(test['userId']==11).select(['movieId','userId'])

In [0]:
# User had 10 ratings in the test data set 
# Realistically this should be some sort of hold out set!
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|      0|    11|
|     32|    11|
|     39|    11|
|     66|    11|
|     72|    11|
|     75|    11|
|     76|    11|
|     77|    11|
|     78|    11|
|     80|    11|
+-------+------+



In [0]:
reccomendations = model.transform(single_user)

In [0]:
reccomendations.orderBy('prediction',ascending=False).show()

+-------+------+------------+
|movieId|userId|  prediction|
+-------+------+------------+
|     80|    11|   3.4978964|
|     75|    11|   3.4293506|
|     66|    11|    2.886164|
|     72|    11|   2.0494604|
|     39|    11|   1.9448571|
|     76|    11|   1.4087913|
|     32|    11|   1.0819743|
|     78|    11|   0.9642752|
|     77|    11|   0.3678526|
|      0|    11|-0.036684018|
+-------+------+------------+

